來自Kelley Blue Book公司的汽車估價資料

In [5]:
import pandas as pd
df = pd.read_excel('http://cdn.sundog-soft.com/Udemy/DataScience/cars.xls')
df.head()

,Price,Mileage,Make,Model,Trim,Type,Cylinder,Liter,Doors,Cruise,Sound,Leather
0,17314.103129,8221,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,1
1,17542.036083,9135,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
2,16218.847862,13196,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
3,16336.913140,16342,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,0
4,16339.170324,19832,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,1


利用里程數、型號和車門數來建立預測汽車銷售的模型，其他特徵都不需要了:

statsmodels套件能進行多元迴歸

In [15]:
import statsmodels.api as sm
df['Model_ord'] = pd.Categorical(df.Model).codes   
df
# 型號為文字形資料 比如 Buick出產的Century型號
# 在進行分析時，所有特徵都應該分配一個數值，因此使用 Categorical() 函數，將資料轉換成數值

,Price,Mileage,Make,Model,Trim,Type,Cylinder,Liter,Doors,Cruise,Sound,Leather,Model_ord
0,17314.103129,8221,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,1,10
1,17542.036083,9135,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0,10
2,16218.847862,13196,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0,10
3,16336.913140,16342,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,0,10
4,16339.170324,19832,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,16507.070267,16229,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,0,0,21
800,16175.957604,19095,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,1,0,21
801,15731.132897,20484,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,1,0,21
802,15118.893228,25979,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,1,0,21


In [13]:
X = df[['Mileage','Model_ord','Doors']]
y = df [['Price']]
X1 = sm.add_constant(X)
est = sm.OLS(y,X1).fit()     # OLS最小平方法
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     11.57
Date:                Wed, 13 Oct 2021   Prob (F-statistic):           1.98e-07
Time:                        13:53:13   Log-Likelihood:                -8519.1
No. Observations:                 804   AIC:                         1.705e+04
Df Residuals:                     800   BIC:                         1.706e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.125e+04   1809.549     17.272      0.000    2.77e+04    3.48e+04
Mileage       -0.1765      0.042     -4.227      0.000      -0.259      -0.095
Model_ord    -39.0387     39.326     -0.993      0.321    -116.234      38.157
Doors      -1652.9303    402.649     -4.105      0.000   -2443.303    -862.558
==============================================================================
Omnibus:                      206.410   Durbin-Watson:                   0.080
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              470.872
Skew:                           1.379   Prob(JB):                    5.64e-103
Kurtosis:                       5.541   Cond. No.                     1.15e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.15e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

> 可以看到R-squared(R平方值)非常小，代表這個模型不好。

> 里程數的標準誤差(std err)是最小的。

> 可以用係數(coef)判斷哪個特徵更重要，但這只有在輸入資料正規化之才有用(特徵值都要在0到1之間)。

> 但如果不是使用正規化的資料，如這個例子，用標準誤差來判定會比較有用。

> 從這個例子中，可以看出里程數是這個模型中最重要的特徵。

其實可以透過切片和切丁，就會發現車門數，事實上對價格影響不大:

In [18]:
y.groupby(df.Doors).mean()

,Price
Doors,
2,23807.135520
4,20580.670749


從表格中可以看出，兩門車的平均售價比四門車還要高，如果車門數和價格之間存在負相關，那還真意想不到，但這只是一個小型資料集，不能代表一切。